In [1]:
using StatsKit, Dates, PlotlyJS

WebIO._IJuliaInit()

In [2]:
dataset = CSV.read("../data/2016-2022.csv", select = [:year,:event_date,:event_type, :sub_event_type, :interaction, :admin1, :location, :latitude, :longitude, :fatalities], DataFrame)

,event_date,year,event_type,sub_event_type,interaction,admin1
,String31,Int64,String31,String,Int64,String
1,18 March 2022,2022,Riots,Mob violence,55,Uttar Pradesh
2,18 March 2022,2022,Protests,Peaceful protest,60,Maharashtra
3,18 March 2022,2022,Protests,Peaceful protest,60,Punjab
4,18 March 2022,2022,Protests,Peaceful protest,60,Madhya Pradesh
5,18 March 2022,2022,Riots,Mob violence,57,Madhya Pradesh
6,18 March 2022,2022,Riots,Mob violence,57,Madhya Pradesh
7,18 March 2022,2022,Protests,Peaceful protest,60,Telangana
8,18 March 2022,2022,Protests,Peaceful protest,60,Telangana
9,18 March 2022,2022,Protests,Peaceful protest,60,Telangana


In [3]:
function timeseries_gen(df::DataFrame, region::String, years::Tuple{Int,Int}, event_type::String = "any", fill::Bool=true, log_scale::Bool=false)
	data = select(df, [:year, :event_date, :admin1, :event_type])
	data = subset(data, :year => y -> y .> years[1]-1 .&& y .< years[2]+1, :admin1 => a -> a .== region)
	if event_type != "any"
		data = subset(data, :event_type => e -> e .== event_type)
	end
	data[!, :time_stamp] = Date.(data.event_date, "d U y")
	count = sort(countmap(data[!,:time_stamp]))
	t = Dates.value.([keys(count)...].-Date(years[1],1,1))
	n = [values(count)...]
	if fill
		l = Int(Dates.value(Date(years[2]+1,1,1)-Day(1)-Date(years[1],1,1)))
		for i in 1:l
			if t[i] != i-1
				insert!(t,i,i-1)
				insert!(n,i,0)
			end
		end
	end
	if log_scale
		n = [val == 0 ? 0 : log(val) for val in n]
	end
	ts    = DataFrame(t = t, n = n)
end

timeseries_gen (generic function with 4 methods)

In [4]:
ts = timeseries_gen(dataset, "Punjab", (2016,2017), "Protests", true, true)

,t,n
,Int64,Real
1,0,1.79176
2,1,1.94591
3,2,1.79176
4,3,1.94591
5,4,1.60944
6,5,1.60944
7,6,0.693147
8,7,1.79176
9,8,1.60944


In [ ]:

    data = scatter(;x = ts.t, y= ts.n,mode="lines")
    plot(data)


data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin and template"

In [6]:
model = loess(ts.t, ts.n, span=0.1);

In [7]:
tp = range(extrema(ts.t)...; step = .1);

In [8]:
np = Loess.predict(model, tp);

In [ ]:
begin
	ts |> @df plot(:t,:n,dpi=600)
	plot!(tp,np)
end

LoadError: LoadError: UndefVarError: @df not defined
in expression starting at In[9]:2

In [ ]:
dif = ts.n .- [np[1:10:end]...]

731-element Vector{Float64}:
  0.14682806120932512
  0.3392137300193834
  0.22068389739284644
  0.4079089432631533
  0.10205051504971263
  0.13031113360233593
 -0.7599410083323389
  0.362556055996627
  0.20200438328061288
  0.5583285163055762
  0.016694095289997657
 -0.2549072496564173
 -0.6456425448034363
  ⋮
  0.8885796810226716
  0.4730767529347413
  0.6172481756884378
 -0.23986249739299947
 -0.24955607465967033
 -0.2591287272406104
  0.019190588901351502
  0.23312464777072717
 -0.2866673037529679
  0.6854620515351011
 -0.016118772335177445
 -0.717388486024286

In [ ]:
plot(ts.t, dif)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin and template"

In [ ]:
histogram(dif)

LoadError: MethodError: no method matching histogram(::Vector{Float64})
[0mClosest candidates are:
[0m  histogram(; kwargs...) at C:\Users\User\.julia\packages\PlotlyBase\xb3Du\src\api.jl:39
[0m  histogram([91m::AbstractDict[39m; kwargs...) at C:\Users\User\.julia\packages\PlotlyBase\xb3Du\src\api.jl:40
[0m  histogram([91m::AbstractDataFrame[39m; kwargs...) at C:\Users\User\.julia\packages\PlotlyBase\xb3Du\src\dataframes_api.jl:504